In [0]:
#import the mlflow autolog. MLflow is used to manage the machine learning lifecycle from initial model development through deployment

import mlflow
mlflow.pyspark.ml.autolog()

In [0]:
#The fault dataset was uploaded using the Data UI 

#Run the databricks utility command to list the datasets in the DBFS. Copy the FaultDataset from the list and read it into Spark Dataframe 
dbutils.fs.ls("/FileStore/tables/")

[FileInfo(path='dbfs:/FileStore/tables/AVITECH-1.csv', name='AVITECH-1.csv', size=580259, modificationTime=1683497136000),
 FileInfo(path='dbfs:/FileStore/tables/AVITECH-2.csv', name='AVITECH-2.csv', size=580259, modificationTime=1683497236000),
 FileInfo(path='dbfs:/FileStore/tables/AVITECH.csv', name='AVITECH.csv', size=580259, modificationTime=1683495554000),
 FileInfo(path='dbfs:/FileStore/tables/AVITECH.xlsx', name='AVITECH.xlsx', size=813945, modificationTime=1683492442000),
 FileInfo(path='dbfs:/FileStore/tables/AdventureWorksLT2019.bak', name='AdventureWorksLT2019.bak', size=8511488, modificationTime=1675183879000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-1.csv', name='FaultDataset-1.csv', size=1703184, modificationTime=1681766199000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-2.csv', name='FaultDataset-2.csv', size=1703184, modificationTime=1682257960000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, mod

Exploratory Data Analysis

In [0]:
#reading the dataset into  Spark DataFrame
faultDatasetDF =spark.read.csv("/FileStore/tables/FaultDataset.csv", header = "true", inferSchema ="true")

In [0]:
#I will also use the print schema to display the names and the datatypes of the columns of the dataset
faultDatasetDF.printSchema()

root
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 4: double (nullable = true)
 |-- 5: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- 9: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)
 |-- 13: double (nullable = true)
 |-- 14: double (nullable = true)
 |-- 15: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 17: double (nullable = true)
 |-- 18: double (nullable = true)
 |-- 19: double (nullable = true)
 |-- 20: double (nullable = true)
 |-- fault_detected: integer (nullable = true)



In [0]:
#df.show() is used here to view the dataset. this gives a bigger picture of what the  faultdataset looks like. 
#Each row contains twenty vibration sensor readings, and the final column identifies whether there was a fault with the machine at the time of the readings.
faultDatasetDF.show(10)

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+
|        1|        2|        3|        4|        5|        6|        7|        8|        9|       10|       11|       12|       13|       14|       15|       16|       17|       18|       19|       20|fault_detected|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+
|0.3503125|0.3496875|     0.35|0.3459375|   0.3475|0.3459375| 0.341875|0.3434375|    0.355|0.3553125|0.3459375|   0.3525|   0.3575|0.3590625|  0.35875|0.3484375|0.3590625|     0.35|0.3559375|0.3490625|             0|
|0.5090625| 0.484375| 0.046875| 0.071875|     0.06|0.0634375|   0.0575|0.0546875|0.0559375| 0.058125|0.0628125| 0.065625|0.0640625|0

In [0]:
#This code perform a basic statistics for the columnns of the faultDF. From the result, it was discovered that no null values for any of the columns. The count is the same for all columns. COUNT = 9292
faultDatasetDF.describe().display()


summary,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,fault_detected
count,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292,9292
mean,0.34162330499354226,0.34263116121394677,0.3421213812957383,0.34213907124407966,0.3428434405940584,0.3428279366659492,0.3427147008179088,0.34306577566724017,0.34317299155187225,0.3439251842983209,0.3441075320167876,0.34398424047567727,0.3441183275936292,0.3448787599547991,0.34489994753551373,0.3458342525828662,0.34563488753766686,0.3457525290572536,0.3465221427034015,0.346670724817048,0.5
stddev,0.28919489486260785,0.2890875372793958,0.28916422490616933,0.28916356333107296,0.2889646554403878,0.2890889899729543,0.2891948159883224,0.28919185608065456,0.2893401858067147,0.289011538534877,0.28920014487495876,0.2890708129465896,0.28911804701463106,0.2889821392646809,0.2891314011350137,0.2888285654988746,0.28892040336707314,0.28915028148431343,0.2887705775702368,0.28900135543931055,0.5000269070362092
min,0.024375,0.024375,0.024375,0.024375,0.024375,0.024375,0.024375,0.024375,0.024375,0.024375,0.024375,0.024375,0.025,0.025,0.025,0.024375,0.024375,0.024375,0.024375,0.025,0
max,1.0809375,1.2134375,1.0809375,1.0809375,1.0809375,1.0809375,1.0809375,1.0809375,1.0809375,1.0809375,1.0809375,1.2134375,1.0809375,1.2134375,1.0809375,1.0809375,1.0809375,1.0809375,1.0809375,1.0809375,1


In [0]:
#the count for the dataset is 9292. the for loop below iterate through the columns and tell the number of null in each of the columns. The result is "0" for all columns
#What this implies is that there is no column with null value.
for col in faultDatasetDF.columns:
    print(col, "\t", "with null values:", faultDatasetDF.filter(faultDatasetDF[col].isNull()).count())

1 	 with null values: 0
2 	 with null values: 0
3 	 with null values: 0
4 	 with null values: 0
5 	 with null values: 0
6 	 with null values: 0
7 	 with null values: 0
8 	 with null values: 0
9 	 with null values: 0
10 	 with null values: 0
11 	 with null values: 0
12 	 with null values: 0
13 	 with null values: 0
14 	 with null values: 0
15 	 with null values: 0
16 	 with null values: 0
17 	 with null values: 0
18 	 with null values: 0
19 	 with null values: 0
20 	 with null values: 0
fault_detected 	 with null values: 0


Distribution of Faults

In [0]:
# Import matplotlib and seaborn libraries
import matplotlib.pyplot as plt 
import seaborn as sns

#Checking the distribution of the dependent variable(fault_detected)

fault_distribution = faultDatasetDF.groupBy('fault_detected').count().toPandas()

#Create bar plot with count values
ax = sns.barplot(x='fault_detected', y='count', data=fault_distribution)

plt.title('Distribution of Faults')

# Add count values to the bars
for i in range(len(fault_distribution)):
    count = fault_distribution['count'][i]
    ax.text(i, count+1, str(count), ha='center')

plt.show()


Correlation of the Sensor Readings

In [0]:
# Import matplotlib and seaborn libraries
import matplotlib.pyplot as plt
import seaborn as sns


# Select the sensor readings columns, exclude the dependent variable
sensor_cols = [str(i) for i in range(1,21)]

# Compute the correlation matrix
corr_matrix = faultDatasetDF.select(sensor_cols).toPandas().corr()

# Plot the heatmap using seaborn
fig, ax = plt.subplots(figsize=(13, 9))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=True)

# Show the plot
plt.show()


DATA PREPROCESSING

In [0]:
#process the data into correct format using Rformula
from pyspark.ml.feature import RFormula
preprocess = RFormula(formula = "fault_detected  ~  . ")
faultDatasetDF = preprocess.fit(faultDatasetDF).transform(faultDatasetDF)


2023/08/22 10:18:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9a816063ae9d4755a289437fb44db2a3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


Splitting the dataset into Training and Test set

In [0]:
#To avoid overfitting issue, the dataset will be splitted into training and test dateset

(trainingDF, testDF) =faultDatasetDF.randomSplit([0.7, 0.3], seed = 100)

Training The DecisionTreeClassifier Model

In [0]:
#importing the decisionTreeClassifier from ml library
from pyspark.ml.classification import DecisionTreeClassifier
#the label and the features are the ones added to the DF during preprocessing. the feature is the vector of all columns except fault_detected 
dt_classifier = DecisionTreeClassifier(labelCol = "label", featuresCol = "features")
#fitting the model with the trained dataset
model = dt_classifier.fit(trainingDF)

2023/08/22 10:18:10 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd8a8e86c0a0245938a75b91e916b7270', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2023/08/22 10:18:10 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.
2023/08/22 10:18:14 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('features', VectorUDT(), True)]. Model signature is not logged.
2023/08/22 10:18:18 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


Evaluating The Model

In [0]:
#predicting the unseen data with the model we trained above
predictions = model.transform(testDF)
predictions.show(10)

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+--------------------+-----+--------------+--------------------+----------+
|        1|        2|        3|        4|        5|        6|        7|        8|        9|       10|       11|       12|       13|       14|       15|       16|       17|       18|       19|       20|fault_detected|            features|label| rawPrediction|         probability|prediction|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+--------------------+-----+--------------+--------------------+----------+
|0.0253125| 0.039375|0.0528125| 0.056875|   0.0725| 0.041875|0.5109375|0.5628125|    0.605|   0.6025|0.6259375|      0.6|0.6090

In [0]:
#Lets view the first 10 rows of our prediction and lets compare it to our label. This shows less than 50 percent accuracy
predictions.select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
+-----+----------+
only showing top 20 rows



Evaluating the Decision Tree Model

In [0]:
#Let's evalaute the accuracy of the whole predictions
#The accuracy is 0.952432, which means that 95% of the predictions made by our model on the test dataset are correct, this is a good result.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol ="label", predictionCol = "prediction", metricName = "accuracy")

accuracy = evaluator.evaluate(predictions)

print ("Accuracy = %g" % (accuracy))

2023/08/22 10:19:12 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.


Accuracy = 0.952432


ParamGridBuilder & TrainValidationSplit for Grid Search

In [0]:
#to identify the optimal set of hyperparameters that predict best (maxDepth,impurity, and maxBins), we will define a grid of values for thes parameters

from pyspark.ml.tuning import ParamGridBuilder 
#Create grid builder
parameters=ParamGridBuilder()\
           .addGrid(dt_classifier.impurity, ["gini", "entropy"]) \
           .addGrid(dt_classifier.maxDepth, [3, 5,  7])   \
           .addGrid(dt_classifier.maxBins,[16,32,64]).build() \



In [0]:
from pyspark.ml.tuning import TrainValidationSplit

#instatiate TrainValidationSplit to split data. fit the decision tree classifer, the grid defined and the evaluator we earlier defined
 
tvs=TrainValidationSplit()   \
    .setSeed(100).setTrainRatio(0.75)   \
    .setEstimatorParamMaps(parameters)  \
    .setEstimator(dt_classifier)        \
    .setEvaluator(evaluator)

In [0]:
#Train model using the grid search

gridsearchmodel = tvs.fit(trainingDF)

2023/08/22 10:19:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ac1d199a3fe3440e9434ecd7d8a12e25', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2023/08/22 10:19:14 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.
2023/08/22 10:20:12 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2023/08/22 10:21:13 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_m

In [0]:
#find the best performing model
bestModel = gridsearchmodel.bestModel
 
 
print("Parameters for the best model:")
print("MaxDepth Parameter: %g" %bestModel.getMaxDepth())
print("Impurity Parameter: %s" %bestModel.getImpurity())
print("MaxBins Parameter: %g" %bestModel.getMaxBins())



Parameters for the best model:
MaxDepth Parameter: 7
Impurity Parameter: entropy
MaxBins Parameter: 64


In [0]:
#make prediction with the test dataset using the bestModel
evaluator.evaluate(bestModel.transform(testDF))

0.961081081081081

Train Random Forest Model

In [0]:
# importing the random forest classifier from ML library
from pyspark.ml.classification import RandomForestClassifier
#the label and the features are the ones added to the DF during preprocessing. The 'features' is the vector of all columns except fault_detected 
rf_classifier = RandomForestClassifier(labelCol = "label", featuresCol = "features")
#fitting the model
rf_model = rf_classifier.fit(trainingDF)

2023/08/22 10:22:04 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a2f1a1e229144e9b9741132d98da191a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2023/08/22 10:22:04 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.
2023/08/22 10:22:15 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


Evaluating the RandomForest Classifier

In [0]:
#use the model to predict the unseen dataset/ test dataset
rf_predictions=rf_model.transform(testDF)
rf_predictions.show(10)

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+--------------------+-----+--------------------+--------------------+----------+
|        1|        2|        3|        4|        5|        6|        7|        8|        9|       10|       11|       12|       13|       14|       15|       16|       17|       18|       19|       20|fault_detected|            features|label|       rawPrediction|         probability|prediction|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+--------------------+-----+--------------------+--------------------+----------+
|0.0253125| 0.039375|0.0528125| 0.056875|   0.0725| 0.041875|0.5109375|0.5628125|    0.605|   0.6025|0.625937

In [0]:
#Lets view the performance of our prediction against our label data
rf_predictions.select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
+-----+----------+
only showing top 20 rows



In [0]:
#Let's evalaute the accuracy of the whole predictions
#The accuracy is  0.967928, which means that about 96.8% of the predictions made by our model on the test dataset are correct, this is a good result.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
rf_evaluator = MulticlassClassificationEvaluator(labelCol ="label", predictionCol = "prediction", metricName = "accuracy")

rf_accuracy = rf_evaluator.evaluate(rf_predictions)

print ("Accuracy = %g" % (rf_accuracy))

2023/08/22 10:23:07 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.


Accuracy = 0.967928


In [0]:
#Create a parameter grid for tuning the model
#to identify the optimal set of hyperparameters(maxDepth,impurity, and maxBins), we will define a grid of values for thes parameters

from pyspark.ml.tuning import ParamGridBuilder 
#Create grid builder
rf_parameters=ParamGridBuilder().addGrid(rf_classifier.impurity, ["gini", "entropy"])\
              .addGrid(rf_classifier.maxDepth, [3, 5,  7])\
              .addGrid(rf_classifier.maxBins,[16,32,64]).build()



In [0]:
from pyspark.ml.tuning import TrainValidationSplit

#Define TrainValidationSplit

rf_tvs=TrainValidationSplit()   \
    .setSeed(100).setTrainRatio(0.75)   \
    .setEstimatorParamMaps(rf_parameters)  \
    .setEstimator(rf_classifier)        \
    .setEvaluator(evaluator)

In [0]:
#Train model using the grid search

rf_gridsearchmodel = rf_tvs.fit(trainingDF)

2023/08/22 10:23:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2d94e389cb7d454dad89423d45e196af', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2023/08/22 10:23:10 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.
2023/08/22 10:24:16 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2023/08/22 10:25:20 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_m

In [0]:
#find the best performing model
rf_bestModel = rf_gridsearchmodel.bestModel
 
 
print("Parameters for the best model:")
print("MaxDepth Parameter: %g" %bestModel.getMaxDepth())
print("Impurity Parameter: %s" %bestModel.getImpurity())
print("MaxBins Parameter: %g" %bestModel.getMaxBins())


Parameters for the best model:
MaxDepth Parameter: 7
Impurity Parameter: entropy
MaxBins Parameter: 64


In [0]:
#make prediction with the test dataset using the bestModel
RFCaccuracy= evaluator.evaluate(rf_bestModel.transform(testDF))

#print the accuracy 
print("The accuracy of the RandomForestClassifier on the test dataset is:", RFCaccuracy )

The accuracy of the RandomForestClassifier on the test dataset is: 0.9704504504504504


Train Gradient-Boosted Trees (GBTs) Model

In [0]:
#Import Gradient-Boosted Trees (GBTs)
from pyspark.ml.classification import GBTClassifier

GBT_classifier = GBTClassifier(labelCol = "label", featuresCol = "features")
#train the model
GBT_model = GBT_classifier.fit(trainingDF)

2023/08/22 10:26:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f464b69fecdf46e9a245729d648b296f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2023/08/22 10:26:14 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.
2023/08/22 10:26:39 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


Evaluating the Model

In [0]:
#use the GBT model to predict unseen dataset 
GBT_predictions=GBT_model.transform(testDF)
GBT_predictions.show(10)

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+--------------------+-----+--------------------+--------------------+----------+
|        1|        2|        3|        4|        5|        6|        7|        8|        9|       10|       11|       12|       13|       14|       15|       16|       17|       18|       19|       20|fault_detected|            features|label|       rawPrediction|         probability|prediction|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+--------------------+-----+--------------------+--------------------+----------+
|0.0253125| 0.039375|0.0528125| 0.056875|   0.0725| 0.041875|0.5109375|0.5628125|    0.605|   0.6025|0.625937

In [0]:
#Lets view the performance of our prediction against our label data
GBT_predictions.select("label","prediction").show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
+-----+----------+
only showing top 10 rows



In [0]:
#Let's evalaute the accuracy of the whole predictions
#The accuracy is  0.961441, which means that about 96.1% of the predictions made by our model on the test dataset are correct, this is a good result.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol ="label", predictionCol = "prediction", metricName = "accuracy")

accuracy = evaluator.evaluate(GBT_predictions)

print ("Accuracy = %g" % (accuracy))

2023/08/22 10:27:40 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.


Accuracy = 0.961441


In [0]:
from pyspark.ml.tuning import ParamGridBuilder
 
#Create grid builder
gbparamGrid =ParamGridBuilder().addGrid(rf_classifier.impurity, ["variance", "entropy"])\
            .addGrid(GBT_classifier.maxDepth, [3, 5,  7])  \
            .addGrid(GBT_classifier.maxBins,[16,32,64]).build()

In [0]:
from pyspark.ml.tuning import TrainValidationSplit

#Define TrainValidationSplit

GBT_tvs=TrainValidationSplit()  \
        .setSeed(100).setTrainRatio(0.75)   \
        .setEstimatorParamMaps(gbparamGrid)   \
        .setEstimator(GBT_classifier)   \
        .setEvaluator(evaluator)    

In [0]:
#Train model using the grid search

GBT_gridsearchmodel = GBT_tvs.fit(trainingDF)

2023/08/22 10:27:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '824d12dde04c444a9cd3690f0227fff0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2023/08/22 10:34:14 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during pyspark.ml autologging: Tuning params should not include params not owned by the tuned estimator, but found a param RandomForestClassifier_1575b5826fb6__impurity


In [0]:
#find the best performing model
GBT_bestModel = GBT_gridsearchmodel.bestModel
 
  
print("Parameters for the best model:")
print("MaxDepth Parameter: %g" %bestModel.getMaxDepth())
print("Impurity Parameter: %s" %bestModel.getImpurity())
print("MaxBins Parameter: %g" %bestModel.getMaxBins())

Parameters for the best model:
MaxDepth Parameter: 7
Impurity Parameter: entropy
MaxBins Parameter: 64


In [0]:
#make prediction with the test dataset using the bestModel
GBTaccuracy = evaluator.evaluate(GBT_bestModel.transform(testDF))

print("The accuracy of the GBTClassifier on the test dataset is:", GBTaccuracy )


The accuracy of the GBTClassifier on the test dataset is: 0.9704504504504504
